In [56]:
import tensorflow as tf

# Print TensorFlow version
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.11.1


In [57]:
#!/usr/bin/env python
# coding: utf-8
#
# **Abstract:**
# A tensorflow LSTM network is used to predict the daily temperature
#  of Amsterdam. Without exogoneous components the best MAE is 1.46
#  degrees on the test set. Cond_rnn is able to get a MAE of 0.87
#  using the temperature in 30 neighbouring
#  cities. A GPU is recommended to speed up calculations,
#  I used a [p3.2xlarge](https://aws.amazon.com/ec2/instance-types/p3/)
#  from AWS for computation.

import numpy as np
# +
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.python.framework.random_seed import set_seed

import temp
from cond_rnn import ConditionalRecurrent



In [58]:

# settings
cells = 200
epochs = 40
test_size = 0.2
validation_split = 0  # we set to 0 for fair comparison with armax
window = 100
cities = 30  # neighbouring cities to include in cond_rnn vector
random_state = 123  # random state  fixed for similar result,
# ideally one should average and report mean
df = temp.read_data(option='daily')

#
# -

# Again, we mainly look at the temperature in Amsterdam.

df_city = (df.droplevel(level=['region', 'country'])
           .unstack(level='date').T.sort_index()
           )
df_city.Amsterdam.head()

# See the notes on ARMA. Here the 30 most correlating
# temperatures are used as exogenous component. In my opinion,
# tensorflow should be better with multicollinearity.


             date      
temperature  1995-01-01    3.111111
             1995-01-02    3.666667
             1995-01-03    0.722222
             1995-01-04   -1.555556
             1995-01-05   -3.444444
Name: Amsterdam, dtype: float64

In [59]:

df_cor = df_city.corr()
df_cor.head()
# One more is grabbed as the most correlating city is Amsterdam itself
top_cities = (df_cor[
                  df_cor.index == 'Amsterdam'].T # Filter for the row for Amsterdams correlation and T for transpose to convert it to a column
              .nlargest(cities + 1, ['Amsterdam']) # Pull the top 31 (cities = 30, so effectively 30+1 = 31)
              .index[0:cities + 1].to_list() # Create a list
              )
df_data = (df_city[top_cities[1:]].shift(1) #subset df_city by the cities in the top_cities list, shift all values down 1 popping the last row's values off, and leaving NA's in the first row
           .assign(Amsterdam=df_city.Amsterdam) #add a column titled 'Amsterdam' with values from df_city.Amsterdam
           .dropna() #drop all NA's (notably the first row)
           )
df_data.columns = df_data.columns.astype(str) # convert column names to string, they were type object

# The dataset is transformed for machine learning.
# Temperature is standard scaled and an input x is generated which contains
#  the previous 100 values for the city of Amsterdam.
# For the other cities, only the previous daily temperature is used.


In [60]:
df_data

city                     Brussels      Paris     London  Copenhagen  \
            date                                                      
temperature 1995-01-02   1.722222   3.000000   1.222222    1.222222   
            1995-01-03   1.888889   1.777778   0.333333   -0.833333   
            1995-01-04   0.777778   2.222222  -0.611111   -4.000000   
            1995-01-05  -1.722222  -0.722222   3.166667   -3.055556   
            1995-01-06  -3.055556  -2.777778   4.166667   -4.833333   
...                           ...        ...        ...         ...   
            2006-04-25  13.444444  13.944444  10.388889    7.833333   
            2006-04-26  14.388889  14.333333  10.777778    7.500000   
            2006-04-27  12.611111  15.000000  11.833333    6.888889   
            2006-04-28  11.388889  12.500000  12.611111    6.944444   
            2006-04-29   8.277778  10.222222  10.000000    6.722222   

city                       Prague     Zurich     Munich    Hamburg       Bern  \
            date                                                                
temperature 1995-01-02   1.222222   1.277778   0.944444  12.888889   1.888889   
            1995-01-03  -1.611111  -2.333333  -1.611111  12.888889  -2.333333   
            1995-01-04  -3.222222  -2.888889  -3.555556  12.888889  -4.000000   
            1995-01-05  -7.277778  -5.444444  -9.833333  -4.666667  -7.888889   
            1995-01-06 -11.333333  -8.277778 -10.222222  -4.666667 -12.611111   
...                           ...        ...        ...        ...        ...   
            2006-04-25  12.000000  14.500000  13.888889  11.333333  14.888889   
            2006-04-26  14.833333  14.666667  14.833333  16.500000  13.888889   
            2006-04-27  15.111111  11.777778  13.833333  12.444444  12.222222   
            2006-04-28  14.611111  11.333333  12.777778  13.833333  12.555556   
            2006-04-29   9.000000   9.944444  10.500000  10.333333  10.166667   

city                       Geneva  ...     Dublin   Helsinki      Minsk  \
            date                   ...                                    
temperature 1995-01-02   3.000000  ...   1.277778   1.611111   1.333333   
            1995-01-03  -1.666667  ...   1.555556  -1.722222  -1.111111   
            1995-01-04  -1.444444  ...   4.500000  -8.611111  -2.888889   
            1995-01-05  -2.888889  ...   7.277778  -2.500000  -9.277778   
            1995-01-06  -7.000000  ...   7.944444  -0.222222 -13.055556   
...                           ...  ...        ...        ...        ...   
            2006-04-25  14.166667  ...   8.888889   4.944444   9.666667   
            2006-04-26  14.166667  ...  11.666667   8.055556  10.944444   
            2006-04-27  13.500000  ...  10.500000   9.555556  11.777778   
            2006-04-28  13.555556  ...  10.111111   9.777778  10.944444   
            2006-04-29  13.222222  ...   9.333333  10.222222  11.833333   

city                      Belfast       Rome      Seoul     Zagreb  Pyongyang  \
            date                                                                
temperature 1995-01-02   1.500000  14.611111  -4.833333  12.055556  -4.444444   
            1995-01-03   2.722222   8.055556   0.666667  12.055556  -0.333333   
            1995-01-04   4.444444   5.333333   0.055556  12.055556   0.277778   
            1995-01-05   7.944444   4.388889  -2.888889  12.055556  -4.277778   
            1995-01-06   9.222222   2.777778  -3.722222  12.055556  -5.000000   
...                           ...        ...        ...        ...        ...   
            2006-04-25   8.388889  15.833333   9.944444  17.777778   9.500000   
            2006-04-26  10.388889  16.777778  10.444444  18.944444  11.000000   
            2006-04-27   9.555556  16.500000  11.944444  17.611111   9.666667   
            2006-04-28   9.666667  15.666667  10.888889  14.833333  10.944444   
            2006-04-29   9.833333  15.833333  12.222222  14.944444  11.111

In [61]:
df_data.Amsterdam

             date      
temperature  1995-01-02     3.666667
             1995-01-03     0.722222
             1995-01-04    -1.555556
             1995-01-05    -3.444444
             1995-01-06    -2.722222
                             ...    
             2006-04-25    13.055556
             2006-04-26    11.888889
             2006-04-27    10.555556
             2006-04-28     7.000000
             2006-04-29     6.944444
Name: Amsterdam, Length: 4136, dtype: float64

In [63]:

ct = ColumnTransformer([
    ('Amsterdam', StandardScaler(), ['Amsterdam']), # Create a scaler named 'Amsterdam' that scales the column Amsterdam according to that columns' mean
    ('Neighbours', StandardScaler(), top_cities[1:]) # Create a scaler named 'Neighbours' that scales all the other columns according to their own means
], remainder='passthrough') #any columns not covered in this list are passed through without scaling (all columns should be scaled)
df_data = pd.DataFrame(ct.fit_transform(df_city[top_cities]), # Apply the transformation and create a dataframe out of it
                       columns=top_cities)

df_data


,Amsterdam,Brussels,Paris,London,Copenhagen,Prague,Zurich,Munich,Hamburg,Bern,...,Riga,Dublin,Helsinki,Minsk,Belfast,Rome,Seoul,Zagreb,Pyongyang,Madrid
0,-1.247673,-1.437018,-1.401960,-1.867586,-1.098770,-0.955524,-1.140456,-1.052270,0.326247,-1.033244,...,-0.614087,-1.906137,-0.473627,-0.635854,-1.918967,-0.211634,-1.566766,-0.045009,-1.362243,-0.914530
1,-1.156645,-1.410750,-1.587962,-2.027869,-1.406764,-1.306892,-1.622811,-1.377826,0.326247,-1.577805,...,-0.792710,-1.843561,-0.839574,-0.884735,-1.653630,-1.246814,-1.051452,-0.045009,-0.998827,-1.329011
2,-1.639093,-1.585866,-1.520325,-2.198169,-1.881241,-1.506690,-1.697020,-1.625532,0.326247,-1.792763,...,-1.213751,-1.180256,-1.595865,-1.065739,-1.279745,-1.676677,-1.108709,-0.045009,-0.944805,-1.743493
3,-2.012307,-1.979875,-1.968419,-1.516967,-1.739730,-2.009629,-2.038379,-2.425269,-2.056190,-2.294333,...,-1.704965,-0.554497,-0.924962,-1.716224,-0.519916,-1.825814,-1.384584,-0.045009,-1.347510,-1.539765
4,-2.321802,-2.190013,-2.281239,-1.336649,-2.006103,-2.512568,-2.416842,-2.474810,-2.056190,-2.903381,...,-1.928244,-0.404315,-0.674898,-2.100859,-0.242518,-2.080222,-1.462662,-0.045009,-1.411353,-1.111233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9260,1.100847,0.988329,0.415778,1.127698,0.507794,0.918441,1.293582,1.127544,NaN,1.073348,...,0.298168,0.684505,0.270466,0.506737,0.806771,0.095411,0.229027,1.095218,0.292775,0.567768
9261,0.390830,0.638098,0.517233,0.125931,0.574387,0.890883,1.115482,0.893993,NaN,1.016026,...,0.655414,-0.229103,0.368052,0.495424,-0.242518,0.437547,0.249848,0.884309,0.425373,0.083036
9262,-0.382908,-0.438861,-0.336681,-0.585324,-0.332947,0.002127,0.173034,0.278266,NaN,0.342489,...,-0.008044,-0.842347,-0.217464,0.959248,-0.724949,0.428774,0.348747,0.772264,0.454839,0.118161
9263,-0.473936,-0.561441,-0.395863,-0.394988,-0.441161,-0.425026,-0.457738,-0.394079,NaN,-0.402699,...,-0.237702,-0.479406,-0.162572,NaN,-0.351065,0.516501,0.197796,-0.295463,0.268220,-0.254169


### Time Lags

Time lags are created as features in the data set since Time Series forecasting assumes that historical data has relevance to future values. This is called autocorrelation. These features are called 'time lags' and are built such that for any row containing a value the model is focusing on, the model can check the appropriately labeled lag column for its previous values.

i.e. if Amsterdam at timestep 5, needed its past 4 values, it can look at the columns x-4,x-3, x-2...

Since we set our window to 100, this does drop the first 100 steps as data as they'll be needed as feature columns for time lags since the first 100 values won't contain all of their previous data. You can see this by looking at the index of df_data in the 3rd chunk below this.


In [66]:


# Creating lags:
# Recall that we want to have time lagged features of up to 100 previous steps.
# We set our window = 100 earlier. For each number from 0 - 100, we create a new column with the original Amsterdam info shifted down 1.

for lag in range(window): #window was set as 100
    df_data.loc[:, f'x-{lag + 1}'] = df_data.Amsterdam.shift(lag + 1)
    # set a column with name 'x-91' = original amsterdam data shifted down 1
df_data = df_data.dropna().sort_index()


C:\Users\dcrai\AppData\Local\Temp\ipykernel_20572\2122746960.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_data.loc[:, f'x-{lag + 1}'] = df_data.Amsterdam.shift(lag + 1)


In [76]:
timestep4 = df_data.Amsterdam.iloc[4]
timestep3 = df_data.Amsterdam.iloc[3]
timestep2 = df_data.Amsterdam.iloc[2]
timestep1 = df_data.Amsterdam.iloc[1]
timestep0 = df_data.Amsterdam.iloc[0]

print(f"Step 0: {timestep0} \n Step 1: {timestep1} \n Step 2: {timestep2} \n Step 3:  {timestep3} \n Step 4: {timestep4}")

Step 0: -0.10982396931146182 
 Step 1: -0.19174908790081238 
 Step 2: -0.3282909522163972 
 Step 3:  -0.34649653412514203 
 Step 4: -0.519449562258216


In [77]:
df_data.loc[105, ['Amsterdam','x-4','x-3','x-2','x-1']]

df_data


,Amsterdam,Brussels,Paris,London,Copenhagen,Prague,Zurich,Munich,Hamburg,Bern,...,x-91,x-92,x-93,x-94,x-95,x-96,x-97,x-98,x-99,x-100
100,-0.109824,-0.071119,0.035321,0.186037,-0.457809,-0.583487,-0.301900,-0.394079,-0.141193,-0.187741,...,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802,-2.012307,-1.639093,-1.156645,-1.247673
101,-0.191749,-0.079874,0.128322,-0.054387,-0.474458,-0.514591,-0.183166,-0.358693,-0.405071,-0.044436,...,-1.147542,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802,-2.012307,-1.639093,-1.156645
102,-0.328291,-0.351303,-0.226771,-0.194635,-0.316298,-0.914186,-0.457738,-0.500239,-0.382453,-0.295220,...,-1.147542,-1.147542,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802,-2.012307,-1.639093
103,-0.346497,-0.421349,-0.218317,0.176019,-0.266353,-0.803953,-0.769413,-0.931955,-0.382453,-0.546005,...,-1.120234,-1.147542,-1.147542,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802,-2.012307
104,-0.519450,-0.403838,-0.387409,-0.184617,-0.441161,-0.638603,-0.695205,-0.811641,-0.382453,-0.460022,...,-0.692403,-1.120234,-1.147542,-1.147542,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4131,0.190568,0.410448,0.263595,-0.214670,-0.108194,0.381054,0.625706,0.596745,0.115145,0.643431,...,-2.157952,-1.629990,-0.728814,-0.492141,-0.892664,-1.092925,-0.865356,-1.648196,-1.857560,-1.511654
4132,0.381727,0.559296,0.322778,-0.144546,-0.158139,0.732423,0.647969,0.717060,0.816305,0.514456,...,-2.094232,-2.157952,-1.629990,-0.728814,-0.492141,-0.892664,-1.092925,-0.865356,-1.648196,-1.857560
4133,0.190568,0.279112,0.424233,0.045789,-0.249705,0.766870,0.262085,0.589668,0.265932,0.299498,...,-1.502551,-2.094232,-2.157952,-1.629990,-0.728814,-0.492141,-0.892664,-1.092925,-0.865356,-1.648196
4134,-0.027899,0.086485,0.043776,0.186037,-0.241381,0.704864,0.202718,0.455199,0.454416,0.342489,...,-1.529859,-1.502551,-2.094232,-2.157952,-1.629990,-0.728814,-0.492141,-0.892664,-1.092925,-0.865356


We can see that for the value at the 5th row and Amsterdam column in our dataframe the value is -0.564.

The historical values acting as features added to the dataframe are accessible under 'x-4', 'x-3','x-2'.. etc. columns.

In [78]:

# The data is split in a train and test set. Shuffle is disabled to enable
#  comparison with ARMAX.

train, test = train_test_split(df_data, test_size=test_size, shuffle=False)



In [89]:

# Libraries are loaded and the data is reshaped.


def create_xy(data):
    'helper function to create x, c and y with proper shapes'
    x = data.filter(like='x-', axis=1).values[:, :, np.newaxis]
    # .filter() - this code filters for all the lag time features
    # .values[] - this converts the dataframe data into a numpy array
    # :, :, np.newaxis - this determines its shape to be rows, columns, 1

    # We change its shape since the LSTM will expect a 3D object.
    
    c = data[top_cities[1:]].to_numpy()
    # this filters the df_data for city columns (non-lagged time series data), not including Amsterdam
     
    y = data.Amsterdam.values[:, np.newaxis]

    # this filters df_data for the Amsterdam time series data and adds an axis since LSTM will expect 2D
    return x, c, y


# create correct shapes for tensorflow
x_train, c_train, y_train = create_xy(train)
x_test, c_test, y_test = create_xy(test)



In [98]:
df_data

,Amsterdam,Brussels,Paris,London,Copenhagen,Prague,Zurich,Munich,Hamburg,Bern,...,x-91,x-92,x-93,x-94,x-95,x-96,x-97,x-98,x-99,x-100
100,-0.109824,-0.071119,0.035321,0.186037,-0.457809,-0.583487,-0.301900,-0.394079,-0.141193,-0.187741,...,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802,-2.012307,-1.639093,-1.156645,-1.247673
101,-0.191749,-0.079874,0.128322,-0.054387,-0.474458,-0.514591,-0.183166,-0.358693,-0.405071,-0.044436,...,-1.147542,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802,-2.012307,-1.639093,-1.156645
102,-0.328291,-0.351303,-0.226771,-0.194635,-0.316298,-0.914186,-0.457738,-0.500239,-0.382453,-0.295220,...,-1.147542,-1.147542,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802,-2.012307,-1.639093
103,-0.346497,-0.421349,-0.218317,0.176019,-0.266353,-0.803953,-0.769413,-0.931955,-0.382453,-0.546005,...,-1.120234,-1.147542,-1.147542,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802,-2.012307
104,-0.519450,-0.403838,-0.387409,-0.184617,-0.441161,-0.638603,-0.695205,-0.811641,-0.382453,-0.460022,...,-0.692403,-1.120234,-1.147542,-1.147542,-0.856253,-0.810739,-1.611784,-1.903074,-2.203466,-2.321802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4131,0.190568,0.410448,0.263595,-0.214670,-0.108194,0.381054,0.625706,0.596745,0.115145,0.643431,...,-2.157952,-1.629990,-0.728814,-0.492141,-0.892664,-1.092925,-0.865356,-1.648196,-1.857560,-1.511654
4132,0.381727,0.559296,0.322778,-0.144546,-0.158139,0.732423,0.647969,0.717060,0.816305,0.514456,...,-2.094232,-2.157952,-1.629990,-0.728814,-0.492141,-0.892664,-1.092925,-0.865356,-1.648196,-1.857560
4133,0.190568,0.279112,0.424233,0.045789,-0.249705,0.766870,0.262085,0.589668,0.265932,0.299498,...,-1.502551,-2.094232,-2.157952,-1.629990,-0.728814,-0.492141,-0.892664,-1.092925,-0.865356,-1.648196
4134,-0.027899,0.086485,0.043776,0.186037,-0.241381,0.704864,0.202718,0.455199,0.454416,0.342489,...,-1.529859,-1.502551,-2.094232,-2.157952,-1.629990,-0.728814,-0.492141,-0.892664,-1.092925,-0.865356


In [97]:
c_train

array([[-0.07111871,  0.03532133,  0.18603677, ..., -0.0450087 ,
        -0.60594413,  0.42024108],
       [-0.07987448,  0.12832192, -0.05438736, ..., -0.0450087 ,
        -0.28181603,  0.55371817],
       [-0.3513031 , -0.22677122, -0.19463477, ..., -0.0450087 ,
         0.13071063,  0.4342913 ],
       ...,
       [-0.95545069, -0.78477473, -1.03611923, ..., -0.53273585,
        -1.42117539, -0.86535401],
       [-1.20936779, -1.44423343, -1.69728559, ..., -0.68432672,
        -1.44081951, -1.24470994],
       [-1.27065813, -1.0130489 , -1.14631362, ..., -1.20500841,
        -0.98409357, -1.13230818]])

In [90]:

# deterministic
set_seed(random_state)

#  As before, I start out by a pure autoregressive model.

model = Sequential(layers=[GRU(cells), Dense(units=1, activation='linear')])
model.compile(optimizer='adam', loss='mae')
history = model.fit(x=x_train, y=y_train, epochs=epochs, batch_size=None,
                    shuffle=True,
                    validation_split=validation_split)


# The final test loss is;


def inverseAms(data):
    return (ct.named_transformers_['Amsterdam']
            .inverse_transform(data)
            )


modelmae = mean_absolute_error(inverseAms(model.predict(x_test)),
                               inverseAms(y_test))
print(f"The MAE is {modelmae:.2f}")

# The above test loss is very similar to ARMA. Let's try to improve on this
#  estimate with an exogenous model.

print("WARNING: Install latest version of cond_rnn via git and not pip!")
model_exog = Sequential(layers=[ConditionalRecurrent(GRU(cells)),
                                Dense(units=1, activation='linear')])
model_exog.compile(optimizer='adam', loss='mae')

# Let's fit a model;

history = model_exog.fit(x=[x_train, c_train], y=y_train, epochs=epochs,
                         batch_size=None, shuffle=True,
                         validation_split=validation_split)

# The test loss for the exogenous model is;

exomae1 = mean_absolute_error(inverseAms(model_exog.predict([x_train,
                                                             c_train])),
                              inverseAms(y_train))
exomae2 = mean_absolute_error(inverseAms(model_exog.predict([x_test,
                                                             c_test])),
                              inverseAms(y_test))

print(f"The train MAE is {exomae1:.2f}")
print(f"The test MAE is {exomae2:.2f}")


Epoch 1/40
101/101 [==============================] - 9s 72ms/step - loss: 0.8175
Epoch 2/40
101/101 [==============================] - 8s 78ms/step - loss: 0.5204
Epoch 3/40
101/101 [==============================] - 8s 78ms/step - loss: 0.4826
Epoch 4/40
101/101 [==============================] - 8s 79ms/step - loss: 0.4730
Epoch 5/40
101/101 [==============================] - 8s 80ms/step - loss: 0.4610
Epoch 6/40
101/101 [==============================] - 9s 84ms/step - loss: 0.3841
Epoch 7/40
101/101 [==============================] - 9s 92ms/step - loss: 0.2940
Epoch 8/40
101/101 [==============================] - 17s 164ms/step - loss: 0.2886
Epoch 9/40
101/101 [==============================] - 17s 166ms/step - loss: 0.2737
Epoch 10/40
101/101 [==============================] - 17s 168ms/step - loss: 0.2625
Epoch 11/40
101/101 [==============================] - 14s 134ms/step - loss: 0.2591
Epoch 12/40
101/101 [==============================] - 16s 158ms/step - loss: 0.2541
Epo